## Target number of units

In [ ]:
import os 
import os.path
import sys
cwd = os.getcwd()
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(parentdir)

In [ ]:
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import utils
import orca
# import datasources
%matplotlib inline

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)
scenarios = utils.yaml_to_dict('../data/scenario_config.yaml', 'scenario')
print(scenarios)

##### from urbansim.parcel

In [ ]:
parcel_capacity_sql = '''
SELECT [parcel_id]
      ,[jurisdiction_id]  
      ,[cap_jurisdiction_id]
      ,[du]
      ,[du_2017]
      ,[capacity_1]
      ,[capacity_2]
      ,[site_id]
  FROM [urbansim].[urbansim].[parcel]'''
capacity_df = pd.read_sql(parcel_capacity_sql,mssql_engine)

##### isam.economic_output.urbansim_housing_units

In [ ]:
units_needed_sql = '''
SELECT [yr]
      ,[demographic_simulation_id]
      ,[household_population]
      ,[householder_rate_id]
      ,[households]
      ,[housing_units1]
      ,[hu_vacancy_rate1]
      ,[housing_units2]
      ,[hu_vacancy_rate2]
      ,[housing_units3]
      ,[housing_units_add]
  FROM [isam].[economic_output].[urbansim_housing_units]
  WHERE demographic_simulation_id = %s'''
units_needed_sql = units_needed_sql % scenarios['demographic_simulation_id']
units_needed_df =  pd.read_sql(units_needed_sql, mssql_engine)

##### urbansim.urbansim.scheduled_development_do_not_use

In [ ]:
sched_dev_do_not_use_sql = '''
SELECT [scenario]
      ,[parcel_id]
      ,[yr]
      ,[site_id]
      ,[res_units]
  FROM [urbansim].[urbansim].[scheduled_development_do_not_use]'''
site_do_not_use = pd.read_sql(sched_dev_do_not_use_sql,mssql_engine)

# Capacity

In [ ]:

total_capacity = int(capacity_df.capacity_2.sum())
print("\nCapacity from urbansim.parcel: {:,}".format(total_capacity))

In [ ]:
# compare to 364,374

# Target units to add 2050 

In [ ]:
units_needed = int(sum(units_needed_df.housing_units_add))
print("\n  Target units to add : {:,}\n\
      (from isam.economic_output.urbansim_housing_units demographic simulation id {:})".\
      format(units_needed,scenarios['demographic_simulation_id']))

# Remaining Capacity at 2050

In [ ]:
print("\n Remaining capacity 2050: {:,}\n       = {:} capacity - {:} units needed\n".\
format(total_capacity - units_needed,total_capacity,units_needed))

## Capacity from scheduled developments

##### urbansim.scheduled_development_do_not_use

In [ ]:
site_from_do_not_use = int(site_do_not_use.res_units.sum())
print("\n  Scheduled developments from scheduled_development_do_not_use: {:,}".format(site_from_do_not_use))

##### urbansim.parcel

In [ ]:
site_from_parcel = int(capacity_df.loc[~capacity_df.site_id.isnull()].capacity_2.sum())
print("\n   Scheduled developments from urbansim.parcel : {:,}".format(int(site_from_parcel)))

# Housing units 2017

In [ ]:
du_2017 = capacity_df.du_2017.sum()
print("\n Housing units 2017 from urbansim.parcel: {:,}\n".format(du_2017))

# Housing units 2050 (forecast)

In [ ]:
du_2017 = capacity_df.du_2017.sum()
print("\n Housing units 2050: {:,}\n\n (= {:,} du 2017 +  {:,} units needed) \n".\
      format(du_2017 + units_needed,du_2017,units_needed))  

In [ ]:
housing_units_db = int(units_needed_df.loc[units_needed_df.yr==2050].iloc[0]['housing_units1'])
print("\n Housing units 2050: {:,}\n\n (from 'housing_units1' column of [isam].[economic_output].[urbansim_housing_units])".\
      format(housing_units_db))

In [ ]:
1203896 - 1192748 - 10947 # housing_units_1 minus du 2017 urbansim.parcel - housing_units_add

#  <span style="color:red">Difference between database table and du 2017 + 'housing_units_add' </span>

In [ ]:
diff_hu_2050 = housing_units_db - (du_2017 + units_needed)
print("\n Difference between database and calculation of housing units 2050: {:,}\n".format(diff_hu_2050))

# Plot (note: increase 2026 and 2027)

In [ ]:
hu_df = units_needed_df[['yr','households','housing_units_add']].copy()

In [ ]:
hu_df['capacity'] = total_capacity

In [ ]:
hu_df['total_housing_units_add'] = hu_df['housing_units_add'].cumsum()

In [ ]:
hu_df.head()

In [ ]:
hu_df['remaining_capacity'] = hu_df['capacity'] - hu_df['total_housing_units_add'] 

In [ ]:
fig, axes = plt.subplots(3, 1)


plot_df = hu_df[['yr','housing_units_add']].copy()
plot_df.rename(columns = {'housing_units_add': 'Housing units to add'}, inplace=True)
plot_df.set_index('yr',inplace=True)
plot_df.plot(style='.-',ax=axes.flat[0],figsize=(10,5),color='b')

plot_df = hu_df[['yr','total_housing_units_add','households']].copy()
plot_df.set_index('yr',inplace=True)
plot_df['Total Housing Units'] = du_2017 + plot_df['total_housing_units_add']
plot_df.rename(columns = {'households': 'Households'}, inplace=True)
del  plot_df['total_housing_units_add']
plot_df.plot(style='.-',ax=axes.flat[1],figsize=(10,10),color=['darkorange','b'])

plot_df = hu_df[['yr','total_housing_units_add','remaining_capacity']].copy()
plot_df.set_index('yr',inplace=True)
plot_df.rename(columns = {'total_housing_units_add': 'Housing units'}, inplace=True)
plot_df.rename(columns = {'remaining_capacity': 'Remaining capacity'}, inplace=True)
plot_df.plot(style='.-',ax=axes.flat[2],figsize=(10,15),color=['b','chocolate'])
plt.axhline(y=0,color='black', lw=2, alpha=0.5,linestyle='--')
# format
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

axes.flat[0].set_xlabel("Simulation Year",size =16)
axes.flat[0].set_ylabel("Housing Units",size =16)
ptitle = 'Housing Units Needed by Year\n demographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[0].set_title(ptitle,size=16) 
axes.flat[0].yaxis.set_major_formatter(tick) 
axes.flat[0].legend(fontsize=14)

# change order of legend in second plot
axes.flat[1].set_xlabel("Simulation Year",size=16)
axes.flat[1].set_ylabel("Total Housing Units",size =16)
ptitle = 'Housing Units and Households \ndemographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[1].set_title(ptitle,size=16) 
axes.flat[1].yaxis.set_major_formatter(tick) 
handles, labels = axes.flat[1].get_legend_handles_labels()
zipped = zip(labels,handles)
zipped.sort(key = lambda t: t[0],reverse=True)
labels, handles = zip(*zipped)
axes.flat[1].legend(handles, labels,fontsize=14)
# axes.flat[1].legend(fontsize=14)

axes.flat[2].set_xlabel("Simulation Year",size =16)
axes.flat[2].set_ylabel("Housing Units (cusum)",size =16)
ptitle = 'Housing Units Needed and Remaining Capacity\n demographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[2].set_title(ptitle,size=16) 
axes.flat[2].yaxis.set_major_formatter(tick) 
axes.flat[2].legend(fontsize=14)

plt.xticks(rotation=25)
plt.tight_layout(pad=0.5, w_pad=0.6, h_pad=2)



plt.show()
fig.savefig('Demographic_simulation_2001.png', format='png', dpi=300)

# Map with folium

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[32.7157,  -117.1611])

In [ ]:
m